In [11]:
import pandas as pd
import numpy as np
from jupDeep import *
import hashlib


'''
    we drop the columns we are not working with
'''
df = pd.read_csv('dataBlock_new.csv')
df = df.drop(['elementHeight', 'elementWidth','element_id','rap_size'], axis=1)


# df.head()

In [12]:
theme_uniq = [ele for ele in df['theme'].unique()]
params_uniq = ['x_orig','y_orig', 'x_end', 'y_end', 'type'] # modifica dupa necesitate globalBlockHeight

var_w_h = [ [27]*2, [50]*2 ]
all_infoBlocks = []

for w, h in var_w_h:
    infoBlocks = {}
    
    col_div = {
           'globalBlockHeight' : h, 
           'x_orig': w, 
           'y_orig': h,
           'x_end': w,
           'y_end': h
          }
    df_new = df.copy()
    
    for col_key in col_div:
        if col_div[col_key] != None:
            df_new[col_key] = df_new[col_key]/col_div[col_key]
            df_new[col_key] = df_new[col_key].apply(np.floor)

    for theme in theme_uniq:
        t = df_new[df_new['theme'] == theme]
        bl = [ele for ele in t['block'].unique()]

        for block in bl:
            block_item = t[t['block']== block][params_uniq]
            block_item = block_item.values.tolist()
            block_item = np.concatenate(block_item).tolist()
            block_item = ''.join(str(block_item))
            way = str(theme) + "/" + str(block)
            infoBlocks[way] = hashlib.sha256(block_item.encode('utf-8')).hexdigest()

    flipped = {}
    for key, value in infoBlocks.items():
        if value not in flipped:
            flipped[value] = [key]
        else:
            flipped[value].append(key)

    res_hash = []
    count = 0
    for hs in flipped:
        if len(flipped[hs])>1:
            count += 1
            res_hash += [flipped[hs]]
    all_infoBlocks += [res_hash]
    print('total for w={}, h={} :  {}'.format(w, h, len(res_hash)))
    


total for w=27, h=27 :  27
total for w=50, h=50 :  35


In [16]:
# df_other = pd.read_csv('dataBlock_new.csv')

small = all_infoBlocks[0]
big   = all_infoBlocks[1]
dif_list = []

name_blocks0 = []
name_blocks1 = []

dist = []
coincidence = []
coincidence_per_dot = []
no_coincidence = []

pp = ['x_orig','y_orig', 'x_end', 'y_end']

for b_big in big:
    if b_big not in small:
        if len(b_big) == 2:
            dif_list += [b_big]
            name_blocks0 += [b_big[0]]
            way0 = b_big[0].split('/')
            theme0 = way0[0]
            block0 = way0[1]

            name_blocks1 += [b_big[1]]
            way1 = b_big[1].split('/')
            theme1 = way1[0]
            block1 = way1[1]

            cond_0 =  (df['theme']==theme0) & (df['block']== block0)
            cond_1 =  (df['theme']==theme1) & (df['block']== block1)

            data0 = df.loc[cond_0, pp ]
            data1 = df.loc[cond_1, pp ]

            d = (((np.array(data0['x_orig']) - np.array(data1['x_orig']))**2 + (np.array(data0['y_end']) - np.array(data1['y_end']))**2).sum())**0.5
            dist += [round(d, 1)]

            cnd = (data0.reset_index(drop=True) == data1.reset_index(drop=True)).to_numpy()
            cnd = np.concatenate(cnd)
            cnd_s = cnd.sum()
            cnd_c = len(cnd)
            dif_c_s = cnd_c-cnd_s
            no_coincidence += [dif_c_s]

            coincidence += [round(100*cnd_s/cnd_c, 1)]
            if dif_c_s != 0:
                coincidence_per_dot += [round(d/dif_c_s, 1)]
            else:
                coincidence_per_dot += ['-']
            
        else:
            print('in acest set sunt mai multe blockuri decit 2 :\n', pare)

df_res = pd.DataFrame ({'name_block 0' : name_blocks0, 'name_block 1' : name_blocks1, 'total diff coord' : dist, 'mean diff coord per param' : coincidence_per_dot, 
                       'no_coincidence' : no_coincidence,  'coincidence %' : coincidence} )

# for index, elem in df_res.iterrows():
#     print(elem)

df_res = df_res.sort_values(by=[ 'coincidence %', 'mean diff coord per param','total diff coord'])
print('numarul total : {}'.format(len(df_res)))
df_res.to_csv('list 27 - 50 - 8 buc.csv', index=False)
df_res.head(50)

numarul total : 8


,name_block 0,name_block 1,total diff coord,mean diff coord per param,no_coincidence,coincidence %
0,mono-blocks/55_links.json,june-local-blocks/50_links.json,37.1,4.6,8,71.4
3,verso-local-blocks/05_images.json,verso-blocks/1_image_-_display_text_with_parag...,28.5,5.7,5,79.2
7,june-blocks/3_columns_-_image_with_boxed_text_...,june-local-blocks/21_links.json,7.1,0.6,12,81.2
1,velvet-local-blocks/17_links.json,velvet-blocks/full_height_image_and_text_with_...,18.0,3.0,6,83.3
2,velvet-local-blocks/42_links.json,velvet-blocks/3_rows_with_image_and_text.json,13.6,1.1,12,88.0
4,verso-local-blocks/03_images.json,verso-blocks/centered_image_with_title_and_key...,10.0,5.0,2,95.0
5,elise-local-blocks/02_images.json,elise-blocks/fullscreen_image_with_title_and_3...,17.0,17.0,1,97.7
6,lyra-local-blocks/12_links.json,lyra-blocks/3_columns_-_full_haight_image_left...,0.0,0.0,1,98.1


In [14]:
import subprocess

path = 'dif_same_blocks'
mkdir(path)

i = 1
for blocks in dif_list:
    way = path + '/' + str(i)
    mkdir(way)
    if len(blocks) == 2:

        for blk in blocks:
            src = '/'.join(['blocks', blk])
            src = src.replace("__", ",_")

            blk_dst = blk.split('/')
            blk_dst = '---'.join(blk_dst)
            blk_dst = blk_dst.replace("__", ",_")


            dst = '/'.join([way, blk_dst])
            command = 'cp {} {}'.format(src, dst)
#             print(src, dst)

            subprocess.call(command, shell=True)

    i += 1
    
print('end code ...')


end code ...


In [17]:
# test


df = pd.read_csv('dataBlock_new.csv')

cond_1 = (df['block']=='81_links.json')  & (df['theme']=='mono-blocks')
cond_2 = (df['block']=='02_links.json')  & (df['theme']=='lyra-local-blocks')

pp = ['x_orig','y_orig', 'x_end', 'y_end','type']

data1 = df.loc[cond_1, pp ]
data2 = df.loc[cond_2, pp ]

cnd = (data1.reset_index(drop=True) == data2.reset_index(drop=True)).to_numpy()

# da = df.loc[cond_1 | cond_2, params_uniq + ['theme','block']]
# da.head(50)

print(cnd)
# data1.head(20)
data2.head(20)
print(data1)
print(data2)

[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True False  True]
 [ True  True  True  True  True]
 [ True  True  True False  True]
 [ True  True  True False  True]
 [ True  True  True  True  True]
 [ True  True  True False  True]
 [ True  True  True False  True]
 [ True  True  True  True  True]
 [ True  True  True False  True]]
      x_orig  y_orig   x_end  y_end    type
1865   850.0   731.0  1040.0  781.0  button
1866   850.0   164.0  1190.0  539.0   image
1867   470.0   731.0   660.0  781.0  button
1868   470.0   164.0   810.0  539.0   image
1869    90.0   731.0   280.0  781.0  button
1870    90.0   164.0   430.0  539.0   image
1871   850.0   657.0  1190.0  688.0    text
1872   850.0   579.0  1190.0  635.0    text
1873   850.0    90.0  1190.0  121.0    text
1874   470.0   657.0   810.0  688.0    text
1875   470.0   579.0